# TSLA stock performance and Twitter mentions

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_grid as altgrid

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

#### Read twitter mentions

In [5]:
twitter = pd.read_csv("data/processed/twitter_mentions_all.csv", parse_dates=["date"])

#### Read stock price

In [6]:
tsla = pd.read_csv("data/processed/tsla_closing_price.csv", parse_dates=["date"])

#### Merge them

In [7]:
merged = pd.merge(twitter, tsla, on="date")[["date", "day_count", "close"]]

In [8]:
merged["day_count_cng"] = merged.day_count.pct_change(1)
merged["close_cng"] = merged.close.pct_change(1)

In [9]:
merged_melt = merged.melt(id_vars="date", value_vars=["day_count_cng", "close_cng"])
merged_melt

,date,variable,value
0,2022-04-20,day_count_cng,NaN
1,2022-04-19,day_count_cng,0.098763
2,2022-04-18,day_count_cng,0.207939
3,2022-04-14,day_count_cng,2.561324
4,2022-04-13,day_count_cng,-0.878878
...,...,...,...
4921,2012-07-13,close_cng,-0.047553
4922,2012-07-12,close_cng,-0.045255
4923,2012-07-11,close_cng,-0.036391
4924,2012-07-10,close_cng,-0.007617


In [10]:
merged

,date,day_count,close,day_count_cng,close_cng
0,2022-04-20,425563,977.200012,NaN,NaN
1,2022-04-19,467593,1028.150024,0.098763,0.052139
2,2022-04-18,564824,1004.289978,0.207939,-0.023207
3,2022-04-14,2011521,985.000000,2.561324,-0.019208
4,2022-04-13,243639,1022.369995,-0.878878,0.037939
...,...,...,...,...,...
2458,2012-07-13,1130,6.850000,2.005319,-0.047553
2459,2012-07-12,164,6.540000,-0.854867,-0.045255
2460,2012-07-11,121,6.302000,-0.262195,-0.036391
2461,2012-07-10,276,6.254000,1.280992,-0.007617


In [11]:
alt.Chart(merged).mark_circle(size=60).encode(
    x="day_count",
    y="close",
)

alt.Chart(...)

In [12]:
src = pd.read_json(
    "https://query1.finance.yahoo.com/v8/finance/chart/TSLA?symbol=TSLA&period1=1341817200&period2=1650985743&useYfid=true&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=IgiovSYKfWc&corsDomain=finance.yahoo.com"
)["chart"]["result"]

In [13]:
df = pd.DataFrame(columns=["datetime", "close"])

In [14]:
df["datetime"] = pd.to_datetime(src[0]["timestamp"], unit="s")
df["close"] = src[0]["indicators"]["quote"][0]["close"]

In [15]:
df["date"] = pd.to_datetime(df["datetime"].dt.date)

In [16]:
alt.Chart(df).mark_line(size=1).encode(
    x=alt.X("date:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("close:Q", title="", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=900, title="Tesla, Inc. (TSLA) stock price at close")

alt.Chart(...)